#Function calling: Criando um assistente de IA capaz de utilizar a API do google maps.

Uma funcionalidade muito desejada e explorada nas LLMs é a de tool use ou function calling, em que o modelo pode utilizar ferramentas externas para realizar uma ação e/ou obter uma resposta. Algumas vantagens dessa técnica são:


*   **Acesso a Dados em Tempo Real**: Através do uso de APIs e ferramentas externas, o assistente de IA pode fornecer informações atualizadas e precisas. Em vez de confiar apenas no conhecimento pré-treinado, o assistente pode fazer chamadas a funções específicas para buscar informações detalhadas e exatas.
* **Automatização de Tarefas**: Funções específicas podem ser chamadas para realizar ações automatizadas. Isso aumenta significativamente as possibilidades de uso.
* **Escalabilidade e Modularidade**: A possibilidade de adicionar novas funcionalidades através de chamadas de funções permite que o assistente seja facilmente escalável e modular. Novas capacidades podem ser incorporadas sem a necessidade de retrainar o modelo inteiro, apenas adicionando novas APIs e funções conforme necessário.

Muitas das mais recentes LLMs já possuem essa funcionalidade integrada. Nos modelos da família Claude 3 da Anthropic, por exemplo, pode-se disponibilizar uma lista de ferramentas diretamente na messages API.
Neste notebook, utilizo o modelo Claude 3.5 sonnet (via Amazon Bedrock) para criar um assistente capaz de utilizar a Places API, do Google Maps, para buscar informações sobre locais próximos.







##1 - Configurando o ambiente
Primeiro, vamos importar as bibliotecas necessárias, configurar o cliente do Amazon Bedrock e a chave da API do Google Maps.

In [ ]:
import boto3
import json
import requests
from math import radians, cos, sin, asin, sqrt

bedrock = boto3.client('bedrock-runtime')
maps_api_key = 'key'

##2 - Listando as ferramentas
Agora, vamos definir a lista de ferramentas que disponibilizaremos ao assistente. Para cada ferramenta, precisamos definir:


*   Nome da ferramenta: Deve ser descritivo e objetivo.
*   Descrição: É de suma importância que a descrição seja a mais detalhada possível, priorizando detalhamento a exemplos de uso.
* Input schema: Define quais são os parâmetros necessários para realizar a ação.



In [ ]:
tools = [
    {
        "name": "search_nearby_places",
        "description": "Retrieves info about nearby places that fit in the user's request. Returns places names, ratings, distance and images.",
        "input_schema": {
            "type": "object",
            "properties": {
                "place_to_search": {
                    "type": "string",
                    "description": "The kind of place the user is asking for."
                },
            },
            "required": ["place_to_search"]
        }
    },
    {
        "name": "get_specific_place_info",
        "description": "Retrieves detailed information about one specific place the users asks for. Returns address, phone number, website, opening hours, rating and price level",
        "input_schema": {
            "type": "object",
            "properties": {
                "place_name": {
                    "type": "string",
                    "description": "The name of the specific place the user wants more information about."
                },
            },
            "required": ["place_name"]
        }
    }
]

##3 - Definição das funções
Após especificar as tools, definimos as funções que serão chamadas quando o assistente identificar a necessidade. Nesse caso, elas são:
* search_nearby_places: Utiliza a API do Google Maps Places para encontrar lugares próximos ao usuário baseado no que ele pedir. Caso o usuário pergunte por "Restaurantes de culinária japonesa", por exemplo, essa função retornará uma lista com os estabelecimentos encontrados próximos à localização base (que defini arbitrariamente).
* get_place_info: Retorna informações mais detalhadas acerca de um estabelecimento em específico, como endereço, contato, rating etc.
Obs: A função haversine é utilizada para calcular a distância entre 2 pares de coordenadas.

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    # Função para encontrar a distãncia entre duas coordenadas.
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Raio da terra em KM.
    return c * r

def search_nearby_places(search):
    #Coordenadas da sede do google em São Paulo
    lat = -23.586675256284725
    lng = -46.682141175631074
    params = {
        'keyword': search,
        'location': str(lat) + ',' + str(lng),
        'radius': 1500,
        'key': maps_api_key
    }

    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    response = requests.get(url, params=params)

    results = []
    for result in response.json()['results']:
        lat1 = result['geometry']['location']['lat']
        lng1 = result['geometry']['location']['lng']
        distance = haversine(lng1, lat1, lng2, lat2)
        results.append({"name": result['name'], "rating": result['rating'], "distance": str(round(distance, 2)) + 'km'})
    return results

def get_place_info(search):
    params = {
        'input': search,
        'inputtype': 'textquery',
        'key': maps_api_key
    }

    url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json'

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: {response.status_code}")

    placeId = data['candidates'][0]['place_id']

    params = {
        'place_id': placeId,
        'fields': 'formatted_address,formatted_phone_number,name,current_opening_hours,website,price_level,rating',
        'key': maps_api_key
    }

    # Define the endpoint
    url = 'https://maps.googleapis.com/maps/api/place/details/json'

    # Make the GET request
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")

##4 - Invocando o modelo
Por fim, definimos a função que invocará o Claude 3.5 Sonnet. Essa função passa o input do usuário e lida com o fluxo de tool use do Claude, que envolve receber um output com "stop_reason" de "tool_use", rodar a função específica da ferramenta requisitada com os parâmetros disponibilizado e enviar de volta ao modelo.

In [ ]:
def chatbot_interaction(user_message):
    print(f"\n{'='*100}\nUser Message: {user_message}\n{'='*100}")

    messages = [
        {"role": "user", "content": user_message}
    ]

    invoke_body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "messages": messages,
        "tools": tools,
        "system":"You are a google maps assistant that can help users get information about places. You will be interacting directly with the end user. That means that you should never reveal what functions or processes you are using behind the scenes."
    }).encode('utf-8')

    invoke = {
        "modelId": "anthropic.claude-3-5-sonnet-20240620-v1:0",
        "contentType": "application/json",
        "accept": "application/json",
        "body": invoke_body
    }

    response = json.loads((bedrock.invoke_model(**invoke)).get('body').read())
    print(f"\n{'='*100}\nAssistente: {response['content'][0]['text']}\n")

    while response['stop_reason'] == "tool_use":
        tool_use = next(block for block in response['content'] if block['type'] == "tool_use")
        tool_name = tool_use['name']
        tool_input = tool_use['input']

        if tool_name == 'search_nearby_places':
            tool_result = search_nearby_places(tool_input['place_to_search'])
        elif tool_name == 'get_specific_place_info':
            tool_result = get_place_info(tool_input['place_name'])

        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response['content']},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use['id'],
                        "content": str(tool_result),
                    }
                ],
            },
        ]

        invoke_body = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 4096,
            "messages": messages,
            "tools": tools,
            "system":"You are a google maps assistant that can help users get information about places. You will be interacting directly with the end user. That means that you should never reveal what functions or processes you are using behind the scenes."
        }).encode('utf-8')

        invoke = {
            "modelId": "anthropic.claude-3-5-sonnet-20240620-v1:0",
            "contentType": "application/json",
            "accept": "application/json",
            "body": invoke_body
        }

        response = json.loads((bedrock.invoke_model(**invoke)).get('body').read())

    final_response = next(
        (block['text'] for block in response['content']),
        None,
    )

    print(f"{final_response}\n{'='*100}")

    return final_response

##5 - Chamando a função
Por fim, podemos chamar o chatbot com a função ``` chatbot_interaction(input) ```.
Aqui estão exemplos de outputs reais do modelo:


```
====================================================================================================
Usuário: Tô morrendo de fome. Sabe se tem algum restaurante japonês barato por aqui?
====================================================================================================

====================================================================================================
Assistente: Claro, posso ajudar você a encontrar um restaurante japonês barato nas proximidades. Vou pesquisar essa informação para você.
====================================================================================================
====================================================================================================
Assistente: Ótimo! Encontrei várias opções de restaurantes japoneses próximos a você. Aqui estão algumas sugestões que podem ser boas opções considerando a proximidade e as avaliações:

1. Mawari: Fica a apenas 470 metros de distância e tem uma avaliação de 4,3 estrelas.
2. Terraço Taiyoo: Está a 520 metros e tem uma excelente avaliação de 4,6 estrelas.
3. Sassá Sushi - Itaim Bibi: Localizado a 550 metros, com uma avaliação de 4 estrelas.
4. JAM Itaim: Um pouco mais distante, a 730 metros, mas tem uma avaliação impressionante de 4,7 estrelas.
5. Temakeria e Cia: A 770 metros de distância, com uma avaliação de 4,1 estrelas.

Essas opções estão relativamente próximas e têm boas avaliações. Lembre-se de que a percepção de "barato" pode variar, então é uma boa ideia verificar os preços ao chegar ao local.

Gostaria de mais informações sobre algum desses restaurantes em particular?
====================================================================================================
```
```
====================================================================================================
Usuário: O Terraço Taiyoo está aberto agora?
====================================================================================================

====================================================================================================
Assistente: Peço desculpas, mas para verificar se o Terraço Taiyoo está aberto agora, preciso obter informações mais detalhadas sobre este local específico. Vou buscar essas informações para você.
====================================================================================================

====================================================================================================
Assistente: Com base nas informações que obtive, posso responder à sua pergunta sobre o Terraço Taiyoo.

Neste momento, o Terraço Taiyoo não está aberto. O restaurante tem horários de funcionamento específicos durante a semana:

- De segunda a sexta-feira: 11:30 às 14:30
- Sábado e domingo: Fechado

Portanto, o Terraço Taiyoo está aberto apenas nos dias úteis, das 11:30 da manhã às 14:30 da tarde.

Outras informações úteis sobre o Terraço Taiyoo:

- Endereço: Av. Dr. Cardoso de Melo, 1671 - 2° andar - Vila Olímpia, São Paulo - SP, 04548-005, Brasil
- Telefone: (11) 3224-9424
- Avaliação: 4,6 estrelas
- Website: Eles têm uma página no Facebook (https://www.facebook.com/terracotaiyoo/?ref=br_rs)

Se você planeja visitar o Terraço Taiyoo, lembre-se de que ele funciona apenas durante o horário de almoço nos dias de semana. É sempre uma boa ideia ligar antes para confirmar o horário de funcionamento ou fazer uma reserva, especialmente se for um grupo grande.

Posso ajudar com mais alguma informação sobre o Terraço Taiyoo ou outro restaurante?
====================================================================================================
```




